In [21]:
pip install streamlit pandas joblib requests matplotlib


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/10/cb/f2ad4230dc2eb1a74edf38f1a38b9b52277f75bef262d8908e60d957e13c/blinker-1.9.0-py3-none-any.whl.metadata
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/72/76/20fa66124dbe6be5cafeb312ece67de6b61dd91a0247d1ea13db4ebb33c2/cachetools-5.5.2-py3-none-any.whl.metadata
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Obtaining dependency information for pillow<12,>=7.1.0 from https://files.pythonhosted.org/packages/3a/c6/fce9255272bcf0c39e15abd2f8fd8429a954cf344469eaceb9d0d1366913/pillow-11.1.0-cp311-cp311-win_amd64.whl.metadata
  Using cached pillow-11.1.0-cp311-cp311-win_amd64.whl.metadata (9.3 kB)
  Obtaining dependency information for protobuf<6,>=3.20 from https://files.py

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 5.29.3 which is incompatible.
tensorflow-intel 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.3 which is incompatible.


In [2]:
pip install streamlit-autorefresh

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for streamlit-autorefresh from https://files.pythonhosted.org/packages/20/82/e378f178498f1d99a672d81df71ebe9693a106cec6a628ee52ce3288cd6d/streamlit_autorefresh-1.0.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/700.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/700.8 kB ? eta -:--:--
    --------------------------------------- 10.2/700.8 kB ? eta -:--:--
    --------------------------------------- 10.2/700.8 kB ? eta -:--:--
   - -------------------------------------- 20.5/700.8 kB 93.1 kB/s eta 0:00:08
   - ------------------------------------- 30.7/700.8 kB 145.2 kB/s eta 0:00:05
   -- ------------------------------------ 41.0/700.8 kB 151.3 kB/s eta 0:00:05
   -- ------------------------------------ 51.2/700.8 kB 174.3 kB/s eta 0:00:04
   ---- ---------------------------------- 81.9/700.8 kB 254.2 kB/s eta 0:00:03
   ---- -

###### final code with Auto refresh code after 30 seconds 

In [26]:
%%writefile ui_app.py

import joblib
import pandas as pd
import requests
import time
import streamlit as st


# Load the trained model
model = joblib.load("rf_model.pkl")  # Ensure this file is in the same directory

# Retrieve Fault Cause Mapping from the Model
fault_cause_mapping = model.fault_cause_mapping  # Now accessible in UI



def fetch_latest_data():
    url = "https://thingspeak.mathworks.com/channels/2815931/feed.json"
    try:
        response = requests.get(url)
        data = response.json()
        latest_entry = data['feeds'][-1]  # Get the most recent data entry
        return {
            "voltage": float(latest_entry["field1"]) if latest_entry["field1"] else 0.0,
            "current": float(latest_entry["field2"]) if latest_entry["field2"] else 0.0,
            "power": float(latest_entry["field3"]) if latest_entry["field3"] else 0.0,
            "energy": float(latest_entry["field4"]) if latest_entry["field4"] else 0.0,
            "frequency": float(latest_entry["field5"]) if latest_entry["field5"] else 50.0,
            "power_factor": float(latest_entry["field6"]) if latest_entry["field6"] else 1.0,
            "temperature": float(latest_entry["field7"]) if latest_entry["field7"] else 25.0
        }
    except Exception as e:
        st.error(f"Error fetching data: {e}")
        return None

# Fetch real-time data
latest_data = fetch_latest_data()


st.set_page_config(layout="wide")

# st.markdown(
#     """
#     <style>
#         body, .stApp {
#             background-color: #f9f9f9; /* Light Off-White Background */
#         }
#     </style>
#     """,
#     unsafe_allow_html=True
# )


# st.markdown("""
#     <h1 style="text-align: left; font-size: 2.5em; margin-top: -60px;">Real-Time Transformer Monitoring & Fault Prediction</h1>
#     <hr style="border: 2px solid #ccc; margin-top: -10px;">
# """, unsafe_allow_html=True)

st.markdown("""
    <div style="
        background-color: black; 
        padding: 15px; 
        border-radius: 8px;
        text-align: left;
        margin-top: -60px;">
        <h1 style="
            font-size: 2.5em; 
            color: white; 
            margin: 0;">
            Real-Time Transformer Monitoring & Fault Prediction
        </h1>
    </div>
    
""", unsafe_allow_html=True)
#<hr style="border: 2px solid #ccc; margin-top: -10px;">



# Embedding ThingSpeak Graphs
st.markdown('<h4 style="margin-bottom: 5px;">Real-Time Sensor Data</h4>', unsafe_allow_html=True)



iframe_links = {
    "Voltage": "https://thingspeak.com/channels/2815931/charts/1?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
    "Current": "https://thingspeak.com/channels/2815931/charts/2?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
    "Power": "https://thingspeak.com/channels/2815931/charts/3?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Energy": "https://thingspeak.com/channels/2815931/charts/4?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
    "Frequency": "https://thingspeak.com/channels/2815931/charts/5?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
    "Power Factor": "https://thingspeak.com/channels/2815931/charts/6?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
    "Temperature": "https://thingspeak.com/channels/2815931/charts/7?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line"
}

col1, col2, col3 = st.columns([1, 1, 1])

# Row 1: Voltage Graph & Voltage Widget
# with col1:
#     st.markdown('<h5 style="margin-bottom: 5px;">Voltage</h5>', unsafe_allow_html=True)
#     st.components.v1.iframe(iframe_links["Voltage"], width=450, height=200)
# with col2:
#     st.markdown('<h5 style="margin-bottom: 5px;">Temperature</h5>', unsafe_allow_html=True)
#     st.components.v1.iframe(iframe_links["Temperature"], width=450, height=200)

with col1:
    st.markdown("""
        <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; overflow: hidden;">
            <h5 style="margin-bottom: 5px;">Voltage</h5>
            <iframe src="{0}" width="450" height="200" style="border: none; overflow: hidden;" scrolling="no"></iframe>
        </div>
    """.format(iframe_links["Voltage"]), unsafe_allow_html=True)

with col2:
    st.markdown("""
        <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; overflow: hidden;">
            <h5 style="margin-bottom: 5px;">Temperature</h5>
            <iframe src="{0}" width="450" height="200" style="border: none; overflow: hidden;" scrolling="no"></iframe>
        </div>
    """.format(iframe_links["Temperature"]), unsafe_allow_html=True)







# Row 2: Temperature Graph & Temperature Widget
with col1:
    st.markdown('<h5 style="margin-bottom: 5px;">Voltage</h5>', unsafe_allow_html=True)
    st.components.v1.iframe("https://thingspeak.com/channels/2815931/widgets/1032359", width=200, height=200)
with col2:
    st.markdown('<h5 style="margin-bottom: 5px;">Temperature</h5>', unsafe_allow_html=True)
    st.components.v1.iframe("https://thingspeak.com/channels/2815931/widgets/1023341", width=200, height=200)

# Remaining Graphs Below
with col1:
    st.markdown("""
        <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; overflow: hidden;">
            <h5 style="margin-bottom: 5px;">Current</h5>
            <iframe src="{0}" width="450" height="200" style="border: none; overflow: hidden;" scrolling="no"></iframe>
        </div>
    """.format(iframe_links["Current"]), unsafe_allow_html=True)

    st.markdown("""
        <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; overflow: hidden;">
            <h5 style="margin-bottom: 5px;">Power</h5>
            <iframe src="{0}" width="450" height="200" style="border: none; overflow: hidden;" scrolling="no"></iframe>
        </div>
    """.format(iframe_links["Power"]), unsafe_allow_html=True)
#     st.subheader("Energy")
#     st.components.v1.iframe(iframe_links["Energy"], width=450, height=200)

with col2:
    st.markdown("""
        <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; overflow: hidden;">
            <h5 style="margin-bottom: 5px;">Frequency</h5>
            <iframe src="{0}" width="450" height="200" style="border: none; overflow: hidden;" scrolling="no"></iframe>
        </div>
    """.format(iframe_links["Frequency"]), unsafe_allow_html=True)

    st.markdown("""
        <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; overflow: hidden;">
            <h5 style="margin-bottom: 5px;">Power Factor</h5>
            <iframe src="{0}" width="450" height="200" style="border: none; overflow: hidden;" scrolling="no"></iframe>
        </div>
    """.format(iframe_links["Power Factor"]), unsafe_allow_html=True)

# Fault Prediction in the 3rd Column
with col3:
    # Fault Prediction Title
    st.markdown("""
        <div style="
            border: 2px solid #ccc; 
            padding: 15px; 
            border-radius: 8px; 
            background-color: black;
            text-align: center;">
            <h4 style="margin-bottom: 5px; color: white ">Fault Prediction</h4>
    """, unsafe_allow_html=True)
 
    # st.markdown('<h4 style="margin-bottom: 5px;">Fault Prediction</h4>', unsafe_allow_html=True)


    
    #prediction above form
        
    if latest_data:
        voltage = latest_data["voltage"]
        current = latest_data["current"]
        power = latest_data["power"]
        energy = latest_data["energy"]
        frequency = latest_data["frequency"]
        power_factor = latest_data["power_factor"]
        temp = latest_data["temperature"]
    else:
        st.warning("Using default values due to an error fetching data.")
        voltage, current, power, energy, frequency, power_factor, temp = 0.0, 0.0, 0.0, 0.0, 50.0, 1.0, 25.0

    # Pass the fetched values to the ML model for prediction
    input_data = pd.DataFrame([[voltage, current, power, energy, frequency, power_factor, temp]],
                              columns=['voltage', 'current', 'power', 'energy', 'frequency', 'power factor', 'temp'])

    # Get the prediction automatically
    predicted_fault = model.predict(input_data)[0]
    predicted_cause = fault_cause_mapping.get(predicted_fault, "Unknown Cause")

#     # Display prediction results (Above the form)
#     st.write("##### **Predicted Fault:**", f":red[{predicted_fault}]")
#     st.write("##### **Cause:**", f":blue[{predicted_cause}]")

    st.markdown(f"""
    <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; background-color: #F4F4FD; margin: 2px 0;">
        <h6 style="color: #999999; margin-bottom: 2px;">Predicted Fault:</h6>
        <p style="font-size: 18px; font-weight: bold; color: #666666; margin-bottom: 2px;">{predicted_fault}</p>
        <h6 style="color: #999999; margin-bottom: 2px;">Cause:</h6>
        <p style="font-size: 18px; font-weight: bold; color: #666666; margin-bottom: 2px;">{predicted_cause}</p>
    </div>
""", unsafe_allow_html=True)

    placeholder = st.empty()  # Create a placeholder for the form

    while True:
        with placeholder.form("fault_form"):
            st.text_input("Voltage (V)", value=voltage, disabled=True)
            st.text_input("Temperature (°C)", value=temp, disabled=True)
            st.text_input("Current (A)", value=current, disabled=True)
            st.text_input("Power (W)", value=power, disabled=True)
            st.text_input("Energy (kWh)", value=energy, disabled=True)
            st.text_input("Frequency (Hz)", value=frequency, disabled=True)
            st.text_input("Power Factor", value=power_factor, disabled=True)

            # Submit button (Runs automatically)
            st.form_submit_button("Refresh Data")

        time.sleep(30)  # Wait for 30 seconds
        st.rerun()  # Refresh the Streamlit app
    st.markdown("</div>", unsafe_allow_html=True)  # Close the styled div

    
    

Overwriting ui_app.py


In [27]:
import numpy as np
import pandas as pd
import sklearn
import joblib
import requests
import time
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# Print versions of the libraries
print("Numpy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Scikit-learn version:", sklearn.__version__)  # Corrected line
print("Joblib version:", joblib.__version__)
print("Requests version:", requests.__version__)
print("Streamlit version:", st.__version__)

Numpy version: 1.24.4
Pandas version: 1.5.3
Scikit-learn version: 1.3.0
Joblib version: 1.2.0
Requests version: 2.32.3
Streamlit version: 1.42.2


###### Predicted fault and cause with manual entering values 


In [5]:
# %%writefile ui_app.py
# import streamlit as st
# import joblib
# import pandas as pd

# # Load the trained model
# model = joblib.load("rf_model.pkl")  # Ensure this file is in the same directory

# # Retrieve Fault Cause Mapping from the Model
# fault_cause_mapping = model.fault_cause_mapping  # Now accessible in UI

# st.set_page_config(layout="wide")
# st.markdown("""
#     <h1 style="text-align: left; font-size: 2.5em; margin-top: -30px;">Real-Time Transformer Monitoring & Fault Prediction</h1>
#     <hr style="border: 2px solid #ccc; margin-top: -10px;">
# """, unsafe_allow_html=True)

# # Embedding ThingSpeak Graphs
# st.header("Real-Time Sensor Data")

# iframe_links = {
#     "Voltage": "https://thingspeak.com/channels/2815931/charts/1?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Current": "https://thingspeak.com/channels/2815931/charts/2?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Power": "https://thingspeak.com/channels/2815931/charts/3?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Energy": "https://thingspeak.com/channels/2815931/charts/4?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Frequency": "https://thingspeak.com/channels/2815931/charts/5?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Power Factor": "https://thingspeak.com/channels/2815931/charts/6?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Temperature": "https://thingspeak.com/channels/2815931/charts/7?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line"
# }

# col1, col2, col3 = st.columns([3, 3, 2])

# # Row 1: Voltage Graph & Voltage Widget
# with col1:
#     st.subheader("Voltage")
#     st.components.v1.iframe(iframe_links["Voltage"], width=400, height=250)
# with col2:
#     st.subheader("Voltage Widget")
#     st.components.v1.iframe("https://thingspeak.com/channels/2815931/widgets/1032359", width=300, height=200)

# # Row 2: Temperature Graph & Temperature Widget
# with col1:
#     st.subheader("Temperature")
#     st.components.v1.iframe(iframe_links["Temperature"], width=400, height=250)
# with col2:
#     st.subheader("Temperature Widget")
#     st.components.v1.iframe("https://thingspeak.com/channels/2815931/widgets/1023341", width=300, height=200)

# # Remaining Graphs Below
# with col1:
#     st.subheader("Current")
#     st.components.v1.iframe(iframe_links["Current"], width=400, height=250)
#     st.subheader("Power")
#     st.components.v1.iframe(iframe_links["Power"], width=400, height=250)
#     st.subheader("Energy")
#     st.components.v1.iframe(iframe_links["Energy"], width=400, height=250)

# with col2:
#     st.subheader("Frequency")
#     st.components.v1.iframe(iframe_links["Frequency"], width=400, height=250)
#     st.subheader("Power Factor")
#     st.components.v1.iframe(iframe_links["Power Factor"], width=400, height=250)

# # Fault Prediction in the 3rd Column
# with col3:
#     st.header("Fault Prediction")
#     voltage = st.number_input("Voltage (V)", min_value=0.0, max_value=500.0, step=0.1)
#     current = st.number_input("Current (A)", min_value=0.0, max_value=100.0, step=0.1)
#     power = st.number_input("Power (W)", min_value=0.0, max_value=5000.0, step=0.1)
#     energy = st.number_input("Energy (kWh)", min_value=0.0, max_value=1000.0, step=0.1)
#     frequency = st.number_input("Frequency (Hz)", min_value=45.0, max_value=65.0, step=0.1)
#     power_factor = st.number_input("Power Factor", min_value=0.0, max_value=1.0, step=0.01)
#     temp = st.number_input("Temperature (°C)", min_value=-10.0, max_value=130.0, step=0.1)

#     if st.button("Predict Fault"):
#         input_data = pd.DataFrame([[voltage, current, power, energy, frequency, power_factor, temp]],
#                                   columns=['voltage', 'current', 'power', 'energy', 'frequency', 'power factor', 'temp'])
        
#         predicted_fault = model.predict(input_data)[0]
#         predicted_cause = fault_cause_mapping.get(predicted_fault, "Unknown Cause")

#         st.write("**Predicted Fault:**", predicted_fault)
#         st.write("**Cause:**", predicted_cause)


Overwriting ui_app.py


###### directly fetching the inputs from thinkspeak json


In [73]:
# %%writefile ui_app.py
# import streamlit as st
# import joblib
# import pandas as pd
# import requests

# # Load the trained model
# model = joblib.load("rf_model.pkl")  # Ensure this file is in the same directory

# # Retrieve Fault Cause Mapping from the Model
# fault_cause_mapping = model.fault_cause_mapping  # Now accessible in UI

# def fetch_latest_data():
#     url = "https://thingspeak.mathworks.com/channels/2815931/feed.json"
#     try:
#         response = requests.get(url)
#         data = response.json()
#         latest_entry = data['feeds'][-1]  # Get the most recent data entry
#         return {
#             "voltage": float(latest_entry["field1"]) if latest_entry["field1"] else 0.0,
#             "current": float(latest_entry["field2"]) if latest_entry["field2"] else 0.0,
#             "power": float(latest_entry["field3"]) if latest_entry["field3"] else 0.0,
#             "energy": float(latest_entry["field4"]) if latest_entry["field4"] else 0.0,
#             "frequency": float(latest_entry["field5"]) if latest_entry["field5"] else 50.0,
#             "power_factor": float(latest_entry["field6"]) if latest_entry["field6"] else 1.0,
#             "temperature": float(latest_entry["field7"]) if latest_entry["field7"] else 25.0
#         }
#     except Exception as e:
#         st.error(f"Error fetching data: {e}")
#         return None

# # Fetch real-time data
# latest_data = fetch_latest_data()


# st.set_page_config(layout="wide")
# st.markdown("""
#     <h1 style="text-align: left; font-size: 2.5em; margin-top: -60px;">Real-Time Transformer Monitoring & Fault Prediction</h1>
#     <hr style="border: 2px solid #ccc; margin-top: -10px;">
# """, unsafe_allow_html=True)



# # Embedding ThingSpeak Graphs
# st.markdown('<h4 style="margin-bottom: 5px;">Real-Time Sensor Data</h4>', unsafe_allow_html=True)

# iframe_links = {
#     "Voltage": "https://thingspeak.com/channels/2815931/charts/1?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Current": "https://thingspeak.com/channels/2815931/charts/2?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Power": "https://thingspeak.com/channels/2815931/charts/3?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
# #     "Energy": "https://thingspeak.com/channels/2815931/charts/4?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Frequency": "https://thingspeak.com/channels/2815931/charts/5?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Power Factor": "https://thingspeak.com/channels/2815931/charts/6?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Temperature": "https://thingspeak.com/channels/2815931/charts/7?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line"
# }

# col1, col2, col3 = st.columns([3, 3, 2])

# # Row 1: Voltage Graph & Voltage Widget
# with col1:
#     st.markdown('<h4 style="margin-bottom: 5px;">Voltage</h4>', unsafe_allow_html=True)
#     st.components.v1.iframe(iframe_links["Voltage"], width=450, height=200)
# with col2:
#     st.markdown('<h4 style="margin-bottom: 5px;">Voltage</h4>', unsafe_allow_html=True)
#     st.components.v1.iframe("https://thingspeak.com/channels/2815931/widgets/1032359", width=200, height=200)

# # Row 2: Temperature Graph & Temperature Widget
# with col1:
#     st.markdown('<h4 style="margin-bottom: 5px;">Temperature</h4>', unsafe_allow_html=True)
#     st.components.v1.iframe(iframe_links["Temperature"], width=450, height=200)
# with col2:
#     st.markdown('<h4 style="margin-bottom: 5px;">Temperature</h4>', unsafe_allow_html=True)
#     st.components.v1.iframe("https://thingspeak.com/channels/2815931/widgets/1023341", width=200, height=200)

# # Remaining Graphs Below
# with col1:
#     st.markdown('<h4 style="margin-bottom: 5px;">Current</h4>', unsafe_allow_html=True)
#     st.components.v1.iframe(iframe_links["Current"], width=450, height=200)
#     st.markdown('<h4 style="margin-bottom: 5px;">Power</h4>', unsafe_allow_html=True)
#     st.components.v1.iframe(iframe_links["Power"], width=450, height=200)
# #     st.subheader("Energy")
# #     st.components.v1.iframe(iframe_links["Energy"], width=450, height=200)

# with col2:
#     st.markdown('<h4 style="margin-bottom: 5px;">Frequency</h4>', unsafe_allow_html=True)
#     st.components.v1.iframe(iframe_links["Frequency"], width=450, height=200)
#     st.markdown('<h4 style="margin-bottom: 5px;">Power Factor</h4>', unsafe_allow_html=True)
#     st.components.v1.iframe(iframe_links["Power Factor"], width=450, height=200)

    
    
# # # Fault Prediction in the 3rd Column with fault button
# # with col3:
# #     st.header("Fault Prediction")

# #     if latest_data:
# #         voltage = st.number_input("Voltage (V)", value=latest_data["voltage"])
# #         current = st.number_input("Current (A)", value=latest_data["current"])
# #         power = st.number_input("Power (W)", value=latest_data["power"])
# #         energy = st.number_input("Energy (kWh)", value=latest_data["energy"])
# #         frequency = st.number_input("Frequency (Hz)", value=latest_data["frequency"])
# #         power_factor = st.number_input("Power Factor", value=latest_data["power_factor"])
# #         temp = st.number_input("Temperature (°C)", value=latest_data["temperature"])
# #     else:
# #         st.warning("Using default values due to an error fetching data.")
# #         voltage = st.number_input("Voltage (V)", value=0.0)
# #         current = st.number_input("Current (A)", value=0.0)
# #         power = st.number_input("Power (W)", value=0.0)
# #         energy = st.number_input("Energy (kWh)", value=0.0)
# #         frequency = st.number_input("Frequency (Hz)", value=50.0)
# #         power_factor = st.number_input("Power Factor", value=1.0)
# #         temp = st.number_input("Temperature (°C)", value=25.0)

# #     # Predict Fault
# #     if st.button("Predict Fault"):
# #         input_data = pd.DataFrame([[voltage, current, power, energy, frequency, power_factor, temp]],
# #                                   columns=['voltage', 'current', 'power', 'energy', 'frequency', 'power factor', 'temp'])

# #         predicted_fault = model.predict(input_data)[0]
# #         predicted_cause = fault_cause_mapping.get(predicted_fault, "Unknown Cause")

# #         st.write("**Predicted Fault:**", predicted_fault)
# #         st.write("**Cause:**", predicted_cause)





# # # Fault Prediction in the 3rd Column without form & fault button
# # with col3:
# #     st.header("Fault Prediction")

# #     if latest_data:
# #         voltage = latest_data["voltage"]
# #         current = latest_data["current"]
# #         power = latest_data["power"]
# #         energy = latest_data["energy"]
# #         frequency = latest_data["frequency"]
# #         power_factor = latest_data["power_factor"]
# #         temp = latest_data["temperature"]
# #     else:
# #         st.warning("Using default values due to an error fetching data.")
# #         voltage, current, power, energy, frequency, power_factor, temp = 0.0, 0.0, 0.0, 0.0, 50.0, 1.0, 25.0

# #     # Convert fetched data into a DataFrame for ML model
# #     input_data = pd.DataFrame([[voltage, current, power, energy, frequency, power_factor, temp]],
# #                               columns=['voltage', 'current', 'power', 'energy', 'frequency', 'power factor', 'temp'])

# #     # Get the prediction automatically
# #     predicted_fault = model.predict(input_data)[0]
# #     predicted_cause = fault_cause_mapping.get(predicted_fault, "Unknown Cause")

# #     # Display prediction results
# #     st.write("### **Predicted Fault:**", f":red[{predicted_fault}]")
# #     st.write("### **Cause:**", f":blue[{predicted_cause}]")


# # Fault Prediction in the 3rd Column
# with col3:
#     st.markdown('<h4 style="margin-bottom: 5px;">Fault Prediction</h4>', unsafe_allow_html=True)

#     # Display fetched sensor values inside a form
#     with st.form("fault_form"):
#         if latest_data:
#             voltage = latest_data["voltage"]
#             current = latest_data["current"]
#             power = latest_data["power"]
#             energy = latest_data["energy"]
#             frequency = latest_data["frequency"]
#             power_factor = latest_data["power_factor"]
#             temp = latest_data["temperature"]
#         else:
#             st.warning("Using default values due to an error fetching data.")
#             voltage, current, power, energy, frequency, power_factor, temp = 0.0, 0.0, 0.0, 0.0, 50.0, 1.0, 25.0

#         st.text_input("Voltage (V)", value=voltage, disabled=True)
#         st.text_input("Current (A)", value=current, disabled=True)
#         st.text_input("Power (W)", value=power, disabled=True)
#         st.text_input("Energy (kWh)", value=energy, disabled=True)
#         st.text_input("Frequency (Hz)", value=frequency, disabled=True)
#         st.text_input("Power Factor", value=power_factor, disabled=True)
#         st.text_input("Temperature (°C)", value=temp, disabled=True)

#         # No need for a submit button, prediction happens automatically
#         st.form_submit_button("Refresh Data")  # Just to refresh values if needed

#     # Automatically pass the fetched values to the ML model
#     input_data = pd.DataFrame([[voltage, current, power, energy, frequency, power_factor, temp]],
#                               columns=['voltage', 'current', 'power', 'energy', 'frequency', 'power factor', 'temp'])

#     # Get the prediction automatically
#     predicted_fault = model.predict(input_data)[0]
#     predicted_cause = fault_cause_mapping.get(predicted_fault, "Unknown Cause")

#     # Display prediction results
#     st.write("### **Predicted Fault:**", f":red[{predicted_fault}]")
#     st.write("### **Cause:**", f":blue[{predicted_cause}]")



Overwriting ui_app.py


###### prediction above form

In [1]:
# %%writefile ui_app.py
# import streamlit as st
# import joblib
# import pandas as pd
# import requests
# import time

# # Load the trained model
# model = joblib.load("rf_model.pkl")  # Ensure this file is in the same directory

# # Retrieve Fault Cause Mapping from the Model
# fault_cause_mapping = model.fault_cause_mapping  # Now accessible in UI



# def fetch_latest_data():
#     url = "https://thingspeak.mathworks.com/channels/2815931/feed.json"
#     try:
#         response = requests.get(url)
#         data = response.json()
#         latest_entry = data['feeds'][-1]  # Get the most recent data entry
#         return {
#             "voltage": float(latest_entry["field1"]) if latest_entry["field1"] else 0.0,
#             "current": float(latest_entry["field2"]) if latest_entry["field2"] else 0.0,
#             "power": float(latest_entry["field3"]) if latest_entry["field3"] else 0.0,
#             "energy": float(latest_entry["field4"]) if latest_entry["field4"] else 0.0,
#             "frequency": float(latest_entry["field5"]) if latest_entry["field5"] else 50.0,
#             "power_factor": float(latest_entry["field6"]) if latest_entry["field6"] else 1.0,
#             "temperature": float(latest_entry["field7"]) if latest_entry["field7"] else 25.0
#         }
#     except Exception as e:
#         st.error(f"Error fetching data: {e}")
#         return None

# # Fetch real-time data
# latest_data = fetch_latest_data()


# st.set_page_config(layout="wide")

# # st.markdown(
# #     """
# #     <style>
# #         body, .stApp {
# #             background-color: #f9f9f9; /* Light Off-White Background */
# #         }
# #     </style>
# #     """,
# #     unsafe_allow_html=True
# # )


# # st.markdown("""
# #     <h1 style="text-align: left; font-size: 2.5em; margin-top: -60px;">Real-Time Transformer Monitoring & Fault Prediction</h1>
# #     <hr style="border: 2px solid #ccc; margin-top: -10px;">
# # """, unsafe_allow_html=True)

# st.markdown("""
#     <div style="
#         background-color: black; 
#         padding: 15px; 
#         border-radius: 8px;
#         text-align: left;
#         margin-top: -60px;">
#         <h1 style="
#             font-size: 2.5em; 
#             color: white; 
#             margin: 0;">
#             Real-Time Transformer Monitoring & Fault Prediction
#         </h1>
#     </div>
    
# """, unsafe_allow_html=True)
# #<hr style="border: 2px solid #ccc; margin-top: -10px;">



# # Embedding ThingSpeak Graphs
# st.markdown('<h4 style="margin-bottom: 5px;">Real-Time Sensor Data</h4>', unsafe_allow_html=True)



# iframe_links = {
#     "Voltage": "https://thingspeak.com/channels/2815931/charts/1?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Current": "https://thingspeak.com/channels/2815931/charts/2?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Power": "https://thingspeak.com/channels/2815931/charts/3?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
# #     "Energy": "https://thingspeak.com/channels/2815931/charts/4?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Frequency": "https://thingspeak.com/channels/2815931/charts/5?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Power Factor": "https://thingspeak.com/channels/2815931/charts/6?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line&update=15",
#     "Temperature": "https://thingspeak.com/channels/2815931/charts/7?bgcolor=%23ffffff&color=%23d62020&dynamic=true&results=60&type=line"
# }

# col1, col2, col3 = st.columns([1, 1, 1])

# # Row 1: Voltage Graph & Voltage Widget
# # with col1:
# #     st.markdown('<h5 style="margin-bottom: 5px;">Voltage</h5>', unsafe_allow_html=True)
# #     st.components.v1.iframe(iframe_links["Voltage"], width=450, height=200)
# # with col2:
# #     st.markdown('<h5 style="margin-bottom: 5px;">Temperature</h5>', unsafe_allow_html=True)
# #     st.components.v1.iframe(iframe_links["Temperature"], width=450, height=200)

# with col1:
#     st.markdown("""
#         <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; overflow: hidden;">
#             <h5 style="margin-bottom: 5px;">Voltage</h5>
#             <iframe src="{0}" width="450" height="200" style="border: none; overflow: hidden;" scrolling="no"></iframe>
#         </div>
#     """.format(iframe_links["Voltage"]), unsafe_allow_html=True)

# with col2:
#     st.markdown("""
#         <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; overflow: hidden;">
#             <h5 style="margin-bottom: 5px;">Temperature</h5>
#             <iframe src="{0}" width="450" height="200" style="border: none; overflow: hidden;" scrolling="no"></iframe>
#         </div>
#     """.format(iframe_links["Temperature"]), unsafe_allow_html=True)







# # Row 2: Temperature Graph & Temperature Widget
# with col1:
#     st.markdown('<h5 style="margin-bottom: 5px;">Voltage</h5>', unsafe_allow_html=True)
#     st.components.v1.iframe("https://thingspeak.com/channels/2815931/widgets/1032359", width=200, height=200)
# with col2:
#     st.markdown('<h5 style="margin-bottom: 5px;">Temperature</h5>', unsafe_allow_html=True)
#     st.components.v1.iframe("https://thingspeak.com/channels/2815931/widgets/1023341", width=200, height=200)

# # Remaining Graphs Below
# with col1:
#     st.markdown("""
#         <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; overflow: hidden;">
#             <h5 style="margin-bottom: 5px;">Current</h5>
#             <iframe src="{0}" width="450" height="200" style="border: none; overflow: hidden;" scrolling="no"></iframe>
#         </div>
#     """.format(iframe_links["Current"]), unsafe_allow_html=True)

#     st.markdown("""
#         <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; overflow: hidden;">
#             <h5 style="margin-bottom: 5px;">Power</h5>
#             <iframe src="{0}" width="450" height="200" style="border: none; overflow: hidden;" scrolling="no"></iframe>
#         </div>
#     """.format(iframe_links["Power"]), unsafe_allow_html=True)
# #     st.subheader("Energy")
# #     st.components.v1.iframe(iframe_links["Energy"], width=450, height=200)

# with col2:
#     st.markdown("""
#         <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; overflow: hidden;">
#             <h5 style="margin-bottom: 5px;">Frequency</h5>
#             <iframe src="{0}" width="450" height="200" style="border: none; overflow: hidden;" scrolling="no"></iframe>
#         </div>
#     """.format(iframe_links["Frequency"]), unsafe_allow_html=True)

#     st.markdown("""
#         <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; overflow: hidden;">
#             <h5 style="margin-bottom: 5px;">Power Factor</h5>
#             <iframe src="{0}" width="450" height="200" style="border: none; overflow: hidden;" scrolling="no"></iframe>
#         </div>
#     """.format(iframe_links["Power Factor"]), unsafe_allow_html=True)

# # Fault Prediction in the 3rd Column
# with col3:
#     # Fault Prediction Title
#     st.markdown("""
#         <div style="
#             border: 2px solid #ccc; 
#             padding: 15px; 
#             border-radius: 8px; 
#             background-color: black;
#             text-align: center;">
#             <h4 style="margin-bottom: 5px; color: white ">Fault Prediction</h4>
#     """, unsafe_allow_html=True)
 
#     # st.markdown('<h4 style="margin-bottom: 5px;">Fault Prediction</h4>', unsafe_allow_html=True)

    
    
    
# #     # prediction below form
# #     # Display fetched sensor values inside a form
# #     with st.form("fault_form"):
# #         if latest_data:
# #             voltage = latest_data["voltage"]
# #             current = latest_data["current"]
# #             power = latest_data["power"]
# #             energy = latest_data["energy"]
# #             frequency = latest_data["frequency"]
# #             power_factor = latest_data["power_factor"]
# #             temp = latest_data["temperature"]
# #         else:
# #             st.warning("Using default values due to an error fetching data.")
# #             voltage, current, power, energy, frequency, power_factor, temp = 0.0, 0.0, 0.0, 0.0, 50.0, 1.0, 25.0

# #         st.text_input("Voltage (V)", value=voltage, disabled=True)
# #         st.text_input("Current (A)", value=current, disabled=True)
# #         st.text_input("Power (W)", value=power, disabled=True)
# #         st.text_input("Energy (kWh)", value=energy, disabled=True)
# #         st.text_input("Frequency (Hz)", value=frequency, disabled=True)
# #         st.text_input("Power Factor", value=power_factor, disabled=True)
# #         st.text_input("Temperature (°C)", value=temp, disabled=True)

# #         # No need for a submit button, prediction happens automatically
# #         st.form_submit_button("Refresh Data")  # Just to refresh values if needed

# #     # Automatically pass the fetched values to the ML model
# #     input_data = pd.DataFrame([[voltage, current, power, energy, frequency, power_factor, temp]],
# #                               columns=['voltage', 'current', 'power', 'energy', 'frequency', 'power factor', 'temp'])

# #     # Get the prediction automatically
# #     predicted_fault = model.predict(input_data)[0]
# #     predicted_cause = fault_cause_mapping.get(predicted_fault, "Unknown Cause")

# #     # Display prediction results
# #     st.write("### **Predicted Fault:**", f":red[{predicted_fault}]")
# #     st.write("### **Cause:**", f":blue[{predicted_cause}]")
    
# #     st.markdown("</div>", unsafe_allow_html=True)  # Close the styled div
    

    
    
#     #prediction above form
        
#     if latest_data:
#         voltage = latest_data["voltage"]
#         current = latest_data["current"]
#         power = latest_data["power"]
#         energy = latest_data["energy"]
#         frequency = latest_data["frequency"]
#         power_factor = latest_data["power_factor"]
#         temp = latest_data["temperature"]
#     else:
#         st.warning("Using default values due to an error fetching data.")
#         voltage, current, power, energy, frequency, power_factor, temp = 0.0, 0.0, 0.0, 0.0, 50.0, 1.0, 25.0

#     # Pass the fetched values to the ML model for prediction
#     input_data = pd.DataFrame([[voltage, current, power, energy, frequency, power_factor, temp]],
#                               columns=['voltage', 'current', 'power', 'energy', 'frequency', 'power factor', 'temp'])

#     # Get the prediction automatically
#     predicted_fault = model.predict(input_data)[0]
#     predicted_cause = fault_cause_mapping.get(predicted_fault, "Unknown Cause")

# #     # Display prediction results (Above the form)
# #     st.write("##### **Predicted Fault:**", f":red[{predicted_fault}]")
# #     st.write("##### **Cause:**", f":blue[{predicted_cause}]")

#     st.markdown(f"""
#     <div style="border: 2px solid #ccc; padding: 5px; border-radius: 8px; background-color: #F4F4FD; margin: 2px 0;">
#         <h6 style="color: #999999; margin-bottom: 2px;">Predicted Fault:</h6>
#         <p style="font-size: 18px; font-weight: bold; color: #666666; margin-bottom: 2px;">{predicted_fault}</p>
#         <h6 style="color: #999999; margin-bottom: 2px;">Cause:</h6>
#         <p style="font-size: 18px; font-weight: bold; color: #666666; margin-bottom: 2px;">{predicted_cause}</p>
#     </div>
# """, unsafe_allow_html=True)

#     # Display fetched sensor values inside a form (Below the prediction)
#     with st.form("fault_form"):
#         st.text_input("Voltage (V)", value=voltage, disabled=True)
#         st.text_input("Temperature (°C)", value=temp, disabled=True)
#         st.text_input("Current (A)", value=current, disabled=True)
#         st.text_input("Power (W)", value=power, disabled=True)
#         st.text_input("Energy (kWh)", value=energy, disabled=True)
#         st.text_input("Frequency (Hz)", value=frequency, disabled=True)
#         st.text_input("Power Factor", value=power_factor, disabled=True)
        

#         # No need for a submit button, prediction happens automatically
#         st.form_submit_button("Refresh Data")  # Just to refresh values if needed

#     st.markdown("</div>", unsafe_allow_html=True)  # Close the styled div

    
    

Overwriting ui_app.py
